## Import models from Hugging Face hub for every supported task
This sample notebook shows how to import and register models from [HuggingFace hub](https://huggingface.co/models) for every supported task. 

### How does import work?
The import process runs as a job in your AzureML workspace using components from the `azureml` system registry. The models are downloaded and converted to MLflow packaged format. You can then register the models to your AzureML Workspace or Registry that makes them available for inference or fine tuning. 

### What models are supported for import?
Any model from Hugging Face hub can be downloaded using the `download_model` component. Following table shows the task supported for mlflow along with the corresponding Huggingface model id for each task:

|Task |Hugging Face Model ID|
|:---|:---|
|fill-mask|bert-base-cased|
|token-classification|Jean-Baptiste/camembert-ner|
|question-answering|deepset/minilm-uncased-squad2|
|summarization|facebook/bart-large-cnn|
|text-generation|distilgpt2|
|text-classification|microsoft/deberta-base-mnli|
|translation|t5-small|
|image-classification|microsoft/resnet-50|
|text-to-image|runwayml/stable-diffusion-v1-5|

Conversion to MLflow will fail if you attempt to download a model that has a task type other than the above with error - `Exception: Unsupported task {task name}`

### Why convert to MLflow?
MLflow is AzureML's recommended model packaging format. 
* **Inference benefits**: AzureML supports no-code-deployment for models packaged as MLflow that enables a seamless inference experience for the models. Learn more about [MLflow model packaging](https://learn.microsoft.com/en-us/azure/machine-learning/concept-mlflow-models) and [no-code-deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=sdk). 
* **Fine tuning benefits**: Foundation models imported and converted to MLflow format can be fine tuned using AzureML's fine tuning pipelines. You can use the no-code UI wizards, or the code based job submission with the SDK or CLI/YAML. AzureML's fine tuning pipelines are built using components. This gives you the flexibility to compose your own fine tuning pipelines containing your own jobs for data transformation, post processing and the AzureML fine tuning components. Learn more about pipelines using [sdk](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python) or [CLI](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipelines-cli).

### What happens if I just download model and register models without converting to MLflow? That's because the task of the model I'm interested in is not among the supported list of tasks.
You can still download and register the model using the outputs of the `download_model` job. You need to [write your own inference code](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python) in this case. It also means that fine tuning is not yet supported if the task type of the model you are interested in is not in the supported list.

### Outline
* Setup pre-requisites such as compute.
* Pick a model to import.
* Configure the import job.
* Run the fine tuning job.
* Register the fine tuned model. 


**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](https://aka.ms/azureml-workspace-configuration)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](https://aka.ms/azureml-sdkv2-install) - check the getting started section


**Motivations** - This notebook explains how to create model importing/publishing pipeline job in workspace using pipeline component registered in a registry and register each model corresponding to every task in workspace

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section, we will connect to the workspace in which the job will be run.

### 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.ai.ml import MLClient, UserIdentityConfiguration
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
from azure.ai.ml.dsl import pipeline

### 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to the workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](https://aka.ms/azureml-workspace-configuration), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 1.3 Get a handle to the workspace

We use the config file to connect to a workspace. The Azure ML workspace should be configured with a computer cluster. [Check this notebook for configure a workspace](https://aka.ms/azureml-workspace-configuration)

If config file is not available user can update following parameters in place holders
- SUBSCRIPTION_ID
- RESOURCE_GROUP
- WORKSPACE_NAME

In [ ]:
# Get a handle to workspace
try:
    ml_client_ws = MLClient.from_config(credential=credential)
except:
    ml_client_ws = MLClient(
        credential,
        subscription_id="<SUBSCRIPTION_ID>",
        resource_group_name="<RESOURCE_GROUP>",
        workspace_name="<WORKSPACE_NAME>",
    )

### 1.4 Compute target setup

#### Create or Attach existing AmlCompute
A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "model-import-cluster"

try:
    _ = ml_client_ws.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS12_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client_ws.begin_create_or_update(compute_config).result()

## 2. Dictionary containing task and model-id as Key-Value pair respectively

In [ ]:
task_model_dict = {
    "fill-mask": "bert-base-cased",
    "token-classification": "Jean-Baptiste/camembert-ner",
    "question-answering": "deepset/minilm-uncased-squad2",
    "summarization": "facebook/bart-large-cnn",
    "text-generation": "distilgpt2",
    "text-classification": "microsoft/deberta-base-mnli",
    "translation": "t5-small",
    "image-classification": "microsoft/resnet-50",
    "text-to-image": "runwayml/stable-diffusion-v1-5",
}

## 3. Start Submitting Jobs for every task parallely

In [ ]:
# Run parallel jobs using multiprocessing
import pipeline_job
from multiprocessing import Pool


num_processors = 9
p = Pool(processes=num_processors)
outputs = p.starmap(
    pipeline_job.create_pipeline_job,
    [
        (
            model_id,
            task,
            compute_name,
        )
        for task, model_id in task_model_dict.items()
    ],
)

## 4. Raise an Error if any import job fails

In [ ]:
failed_model_lists = []
for output in outputs:
    for model_id, job_status in output.items():
        if not job_status:
            failed_model_lists.append(model_id)

if failed_model_lists:
    raise Exception(
        f"Import Job failed for given models {', '.join(failed_model_lists)}"
    )